In [2]:
pip install vcf

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement vcf (from versions: none)
ERROR: No matching distribution found for vcf


In [13]:
import pandas as pd


def read_vcf(file: str) -> pd.DataFrame:
    num_header = 0
    with open(file) as f:
        for line in f.readlines():
            if line.startswith("##"):
                num_header += 1
            else:
                break
    vcf = pd.read_csv(file, sep="\t", skiprows=num_header)
    vcf = vcf.rename({"#CHROM": "CHROM"}, axis=1)
    return vcf

In [14]:
import pandas as pd

df = read_vcf('GBS_filtered_SNPs.vcf')

In [16]:
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,102,...,cham6,clear_white,dharwar_dry,hidhab,klein_chamaco,opata,pavon,pbw343,rac875,vorobey
0,3929455_1al,1623,.,T,C,245.53,.,AC=18;AF=0.196;AN=92;BaseQRankSum=0.079;DP=48;...,GT:AD:DP:GQ:PL,"0/0:1,0:1:3:0,3,41",...,"0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39",./.,./.,"1/1:0,1:1:3:39,3,0","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39"
1,3929455_1al,1625,.,A,C,417.94,.,AC=28;AF=0.304;AN=92;BaseQRankSum=-1.418;DP=48...,GT:AD:DP:GQ:PL,"0/0:1,0:1:3:0,3,41",...,"0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39",./.,./.,"1/1:0,1:1:3:39,3,0","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39"
2,3967255_1al,9694,.,C,G,654.37,.,AC=41;AF=0.477;AN=86;BaseQRankSum=1.762;DP=45;...,GT:AD:DP:GQ:PL,./.,...,./.,"0/0:1,0:1:3:0,3,37","1/1:0,1:1:3:39,3,0",./.,"1/1:0,1:1:3:37,3,0","0/0:1,0:1:3:0,3,39","1/1:0,1:1:3:39,3,0","0/0:1,0:1:3:0,3,39","0/0:1,0:1:3:0,3,39","0/1:1,1:2:33:33,0,33"
3,3967255_1al,9864,.,A,G,663.37,.,AC=41;AF=0.477;AN=86;BaseQRankSum=0.141;DP=45;...,GT:AD:DP:GQ:PL,./.,...,./.,"0/0:1,0:1:3:0,3,39","1/1:0,1:1:3:40,3,0",./.,"1/1:0,1:1:3:40,3,0","0/0:1,0:1:3:0,3,40","1/1:0,1:1:3:40,3,0","0/0:1,0:1:3:0,3,40","0/0:1,0:1:3:0,3,40","0/1:1,1:2:29:34,0,29"
4,3967255_1al,9908,.,C,T,752.26,.,AC=43;AF=0.489;AN=88;BaseQRankSum=0.125;DP=45;...,GT:AD:DP:GQ:PL,./.,...,./.,"0/0:1,0:1:3:0,3,41","1/1:0,1:1:3:42,3,0",./.,"1/1:0,1:1:3:42,3,0","0/0:1,0:1:3:0,3,42","1/1:0,1:1:3:42,3,0","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/1:1,1:2:32:36,0,32"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210647,3848042_7ds,176,.,A,G,33.32,.,AC=4;AF=0.040;AN=100;BaseQRankSum=0.218;DP=52;...,GT:AD:DP:GQ:PL,./.,...,./.,"0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42"
210648,3848042_7ds,195,.,G,C,32.32,.,AC=4;AF=0.040;AN=100;BaseQRankSum=-1.577;DP=51...,GT:AD:DP:GQ:PL,./.,...,./.,"0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","1/1:0,1:1:3:41,3,0","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42"
210649,3848042_7ds,199,.,A,C,33.32,.,AC=4;AF=0.040;AN=100;BaseQRankSum=0.655;DP=51;...,GT:AD:DP:GQ:PL,./.,...,./.,"0/0:1,0:1:3:0,3,42","1/1:0,1:1:3:42,3,0","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42","0/0:1,0:1:3:0,3,42"
210650,3848042_7ds,305,.,A,G,59.12,.,AC=6;AF=0.068;AN=88;BaseQRankSum=2.063;DP=82;D...,GT:AD:DP:GQ:PL,./.,...,"0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45","0/0:2,0:2:3:0,3,45"
